###  Import Libraries and enviornment variables

In [1]:
### LLMs
import os
from dotenv import load_dotenv

# Load environment variables from '.env' file
load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY') # For LLM -- llama-3.1-8b (small) & mixtral-8x7b-32768 (large)
os.environ['COHERE_API_KEY'] = os.getenv('COHERE_API_KEY') # For embedding

### Create Vectorstore

In [2]:
### Build Index
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_cohere import CohereEmbeddings
from langchain.document_loaders import CSVLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

# Set embeddings
# embedding_model = CohereEmbeddings(model="embed-english-v3.0")

EMBEDDING_MODEL_NAME = "dangvantuan/vietnamese-document-embedding"

embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda", "trust_remote_code": True},
    encode_kwargs={"normalize_embeddings": True},  # set True for cosine similarity
)

USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\hello\AppData\Local\Temp\ipykernel_4664\2694590058.py:15: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In [3]:

csv_file_path = fr"C:\htN\UIT\learnin\RAG_Techniques\data\alqac_2024\training\EDA\law_articles.csv"

# Load CSV file
loader = CSVLoader(file_path=csv_file_path)
raw_docs = loader.load()

for doc in raw_docs:
    doc.page_content = doc.page_content.replace('\n', ' ')

# print(raw_docs)

# Tạo lại documents với metadata mong muốn
docs_list = []
for doc in raw_docs:
    # Parse nội dung từ page_content
    content = doc.page_content
    
    # Tách các phần thông tin
    parts = content.split(' content: ')
    header = parts[0].split(' article_id: ')
    id_law = header[0].replace('id_law: ', '')
    article_id = header[1]
    content = parts[1] if len(parts) > 1 else ''
    
    # Tạo document mới với metadata đã parse
    new_doc = Document(
        page_content=content.replace('\n', ' '),  # Thay thế \n bằng khoảng trắng trong content
        metadata={
            'title': f"Điều {article_id} - {id_law}",
            'source': id_law,
            'article_id': article_id
        }
    )
    docs_list.append(new_doc)


In [4]:
# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

print(f"Number of text splits generated: {len(doc_splits)}")

Number of text splits generated: 4615


In [4]:


# Add to vectorstore
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag",
    embedding=embedding_model,
)

retriever = vectorstore.as_retriever(
                search_type="similarity",
                search_kwargs={'k': 4}, # number of documents to retrieve
            )

### Question

In [5]:
question = "Người nghiện ma túy từ đủ 18 tuổi trở lên bị áp dụng biện pháp xử lý hành chính đưa vào cơ sở cai nghiện bắt buộc theo quy định của Luật Xử lý vi phạm hành chính khi bị phát hiện sử dụng chất ma túy một cách trái phép trong thời gian cai nghiện ma túy tự nguyện, đúng hay sai?"
question_type = "Đúng/Sai"

### Retrieve docs

In [6]:
docs = retriever.invoke(question)

### Check what our doc looklike

In [7]:
print(f"Title: {docs[0].metadata['title']}\n\nSource: {docs[0].metadata['source']}\n\nContent: {docs[0].page_content}\n")

Title: Điều 32 - Luật Phòng, chống ma túy

Source: Luật Phòng, chống ma túy

Content: Đối tượng bị áp dụng biện pháp xử lý hành chính đưa vào cơ sở cai nghiện bắt buộc  Người nghiện ma túy từ đủ 18 tuổi trở lên bị áp dụng biện pháp xử lý hành chính đưa vào cơ sở cai nghiện bắt buộc theo quy định của Luật Xử lý vi phạm hành chính khi thuộc một trong các trường hợp sau đây:  1. Không đăng ký, không thực hiện hoặc tự ý chấm dứt cai nghiện ma túy tự nguyện;  2. Trong thời gian cai nghiện ma túy tự nguyện bị phát hiện sử dụng trái phép chất ma túy;  3. Người nghiện ma túy các chất dạng thuốc phiện không đăng ký, không thực hiện hoặc tự ý chấm



### Check document relevancy

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
from langchain_community.llms import HuggingFaceHub

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
# LLM
llm = ChatGroq(model="deepseek-r1-distill-llama-70b", temperature=0)


In [11]:
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader

### Filter out the non-relevant docs

In [12]:
docs_to_use = []
for doc in docs:
    print(doc.page_content, '\n', '-'*50)
    res = retrieval_grader.invoke({"question": question, "document": doc.page_content})
    print(res,'\n')
    if res.binary_score == 'yes':
        docs_to_use.append(doc)

Đối tượng bị áp dụng biện pháp xử lý hành chính đưa vào cơ sở cai nghiện bắt buộc  Người nghiện ma túy từ đủ 18 tuổi trở lên bị áp dụng biện pháp xử lý hành chính đưa vào cơ sở cai nghiện bắt buộc theo quy định của Luật Xử lý vi phạm hành chính khi thuộc một trong các trường hợp sau đây:  1. Không đăng ký, không thực hiện hoặc tự ý chấm dứt cai nghiện ma túy tự nguyện;  2. Trong thời gian cai nghiện ma túy tự nguyện bị phát hiện sử dụng trái phép chất ma túy;  3. Người nghiện ma túy các chất dạng thuốc phiện không đăng ký, không thực hiện hoặc tự ý chấm 
 --------------------------------------------------
binary_score='yes' 

Cai nghiện ma túy cho người từ đủ 12 tuổi đến dưới 18 tuổi  1. Người nghiện ma túy từ đủ 12 tuổi đến dưới 18 tuổi bị đưa vào cơ sở cai nghiện bắt buộc khi thuộc một trong các trường hợp sau đây:  a) Không đăng ký, không thực hiện hoặc tự ý chấm dứt cai nghiện ma túy tự nguyện;  b) Trong thời gian cai nghiện ma túy tự nguyện bị phát hiện sử dụng trái phép chất ma t

In [13]:
print(docs_to_use)

[Document(metadata={'article_id': '32', 'source': 'Luật Phòng, chống ma túy', 'title': 'Điều 32 - Luật Phòng, chống ma túy'}, page_content='Đối tượng bị áp dụng biện pháp xử lý hành chính đưa vào cơ sở cai nghiện bắt buộc  Người nghiện ma túy từ đủ 18 tuổi trở lên bị áp dụng biện pháp xử lý hành chính đưa vào cơ sở cai nghiện bắt buộc theo quy định của Luật Xử lý vi phạm hành chính khi thuộc một trong các trường hợp sau đây:  1. Không đăng ký, không thực hiện hoặc tự ý chấm dứt cai nghiện ma túy tự nguyện;  2. Trong thời gian cai nghiện ma túy tự nguyện bị phát hiện sử dụng trái phép chất ma túy;  3. Người nghiện ma túy các chất dạng thuốc phiện không đăng ký, không thực hiện hoặc tự ý chấm')]


### Generate Result

In [16]:
from langchain_core.output_parsers import StrOutputParser

# Prompt
system = """You are a professional in answering legal law questions. Use the provided documents to answer those question. Always read the documents carefully before answer. Please answer the question based on the type of the question. You must always answer the question in Vietnamese. You are provided with the following:
1. A question.
2. The type of the question
3. A set of documents that were referenced in generating the answer.

If the question is marked "Trắc nghiệm", answering the only choice that you think are right based on the document you are provided.
Example: 
With the question: "Thông báo nghỉ hưu đối với viên chức được cơ quan, đơn vị, tổ chức ban hành trong thời gian bao lâu trước khi đến thời hạn viên chức nghỉ hưu? ",Luật Viên chức,46,"'A': '06 tháng', 'B': '04 tháng', 'C': '03 tháng', 'D': '02 tháng'", you will answer "B: 04 tháng"

If the question is marked "Đúng/Sai", choose Đúng or Sai based on the knowledge you've been provided, provide the name of the law, the id of the law (This is usually behind the "Điều" part), and your answer, based on the example answer, not answer anything else.
Example:
For the question: "Việc lựa chọn nơi cư trú của vợ chồng do phong tục tập quán của gia đình nhà chồng quyết định, đúng hay sai?", you will answer: "Luật Hôn nhân và gia đình,20,,Sai"

If the question is marked "Tự luận", answer based on the documents that you've been provided, provide the name of the law, the id of the law (This is usually behind the "Điều" part), and your answer, based on the example answer, not answer anything else.
Example:
With the question: "Cơ quan nào có trách nhiệm thống nhất quản lý nhà nước về điện ảnh?", you should answer: "Luật Điện ảnh,45,,Chính phủ"

Question: {question} 
Type of question: {question_type}
Context: {documents} 
Answer:"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved documents: \n\n <docs>{documents}</docs> \n\n User question: <question>{question}</question> \n\n Question Type: <question_type>{question_type}</question_type>"),
    ]
)



# Post-processing
def format_docs(docs):
    return "\n".join(f"<doc{i+1}>:\nTitle:{doc.metadata['title']}\nSource:{doc.metadata['source']}\nContent:{doc.page_content}\n</doc{i+1}>\n" for i, doc in enumerate(docs))

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"documents":format_docs(docs_to_use), "question": question, "question_type": question_type})
print(generation)

<think>
Được rồi, mình cần phân tích câu hỏi này. Câu hỏi là về việc người nghiện ma túy từ đủ 18 tuổi trở lên có bị áp dụng biện pháp xử lý hành chính đưa vào cơ sở cai nghiện bắt buộc khi bị phát hiện sử dụng ma túy trái phép trong thời gian cai nghiện tự nguyện hay không.

Mình xem qua tài liệu được cung cấp, đó là Điều 32 của Luật Phòng, chống ma túy. Điều này quy định rõ các trường hợp áp dụng biện pháp xử lý hành chính đưa vào cơ sở cai nghiện bắt buộc. Cụ thể, Khoản 2 của Điều 32 nói rằng nếu trong thời gian cai nghiện tự nguyện mà bị phát hiện sử dụng trái phép chất ma túy, thì người đó sẽ bị áp dụng biện pháp này.

Câu hỏi hoàn toàn khớp với trường hợp được nêu trong Khoản 2 của Điều 32. Do đó, câu trả lời là đúng.
</think>

Luật Phòng, chống ma túy,32,,Đúng


### Check for Hallucinations

In [17]:
# Data model
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in 'generation' answer."""

    binary_score: str = Field(
        ...,
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )

# LLM with function call
structured_llm_grader = llm.with_structured_output(GradeHallucinations)

# Prompt
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n 
    Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n <facts>{documents}</facts> \n\n LLM generation: <generation>{generation}</generation>"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader

response = hallucination_grader.invoke({"documents": format_docs(docs_to_use), "generation": generation})
print(response)

binary_score='yes'


### Highlight used docs

In [27]:
from typing import List
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate

# Data model
class HighlightDocuments(BaseModel):
    """Return the specific part of a document used for answering the question."""

    id: List[str] = Field(
        ...,
        description="List of id of docs used to answers the question"
    )

    title: List[str] = Field(
        ...,
        description="List of titles used to answers the question"
    )

    source: List[str] = Field(
        ...,
        description="List of sources used to answers the question"
    )

    segment: List[str] = Field(
        ...,
        description="List of direct segements from used documents that answers the question"
    )


# parser
parser = PydanticOutputParser(pydantic_object=HighlightDocuments)

# Prompt
system = """You are an advanced assistant for document search and retrieval. You are provided with the following:
1. A question.
2. A generated answer based on the question.
3. A set of documents that were referenced in generating the answer.

You are a professional in answering legal law questions. Use the provided documents to answer those question. Please answer the question based on the type of the question. You must always answer the question in Vietnamese. You are provided with the following:
1. A question.
2. A generated answer based on the question.
3. A set of documents that were referenced in generating the answer.

If the question is marked "Trắc nghiệm", answering the only choice that you think are right.
Example: 
With the question: "Thông báo nghỉ hưu đối với viên chức được cơ quan, đơn vị, tổ chức ban hành trong thời gian bao lâu trước khi đến thời hạn viên chức nghỉ hưu? ",Luật Viên chức,46,"'A': '06 tháng', 'B': '04 tháng', 'C': '03 tháng', 'D': '02 tháng'", you will answer "B: 04 tháng"

If the question is marked "Đúng/Sai", choose Đúng or Sai based on the knowledge you've been provided, and provide the law that you based on.
Example:
For the question: "Việc lựa chọn nơi cư trú của vợ chồng do phong tục tập quán của gia đình nhà chồng quyết định, đúng hay sai?", you will answer: "Luật Hôn nhân và gia đình,20,,Sai"

If the question is marked "Tự luận", answer based on the documents that you've been provided, provide the name of the law, the id of the law, and your answer.
Example:
With the question: "Cơ quan nào có trách nhiệm thống nhất quản lý nhà nước về điện ảnh?", you should answer: "Luật Điện ảnh,45,,Chính phủ"

Question: {question} 
Type of question: {question_type}
Context: {documents} 
Answer:

Ensure that:
- (Important) Each segment is an exact match to a part of the document and is fully contained within the document text.
- The relevance of each segment to the generated answer is clear and directly supports the answer provided.
- (Important) If you didn't used the specific document don't mention it.

Used documents: <docs>{documents}</docs> \n\n User question: <question>{question}</question> \n\n Generated answer: <answer>{generation}</answer>

<format_instruction>
{format_instructions}
</format_instruction>
"""


prompt = PromptTemplate(
    template= system,
    input_variables=["documents", "question", "generation"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Chain
doc_lookup = prompt | llm | parser

# Run
lookup_response = doc_lookup.invoke({"documents":format_docs(docs_to_use), "question": question, "question_type": question_type, "generation": generation})

In [28]:
for id, title, source, segment in zip(lookup_response.id, lookup_response.title, lookup_response.source, lookup_response.segment):
    print(f"ID: {id}\nTitle: {title}\nSource: {source}\nText Segment: {segment}\n")

ID: doc1
Title: Điều 32 - Luật Phòng, chống ma túy
Source: Luật Phòng, chống ma túy
Text Segment: Người nghiện ma túy từ đủ 18 tuổi trở lên bị áp dụng biện pháp xử lý hành chính đưa vào cơ sở cai nghiện bắt buộc theo quy định của Luật Xử lý vi phạm hành chính khi thuộc một trong các trường hợp sau đây:  2. Trong thời gian cai nghiện ma túy tự nguyện bị phát hiện sử dụng trái phép chất ma túy;



In [ ]:
# import pandas as pd

# df = pd.read_csv(fr"C:\htN\UIT\learnin\RAG_Techniques\data\alqac_2024\training\EDA\train_alqac.csv")

# df['new_Answer'] = (df['law_id'].fillna('') + ',' + 
#                    df['article_id'].astype(str).replace('nan', '') + ',' + 
#                    df['choices'].fillna('') + ',' + 
#                    df['Answer'].fillna(''))

# df.to_csv(fr"C:\htN\UIT\learnin\RAG_Techniques\data\alqac_2024\training\EDA\train_alqac_compare.csv")

In [24]:
import pandas as pd
import time

def process_questions(csv_file):
    # Đọc file CSV
    df = pd.read_csv(csv_file)
    results = []
    
    for index, row in df.iterrows():
        question = row['Question']  # Giả sử cột chứa câu hỏi tên là 'question'
        question_type = row['question_type']

        start = time.time()
        # 1. Retrieve docs
        docs = retriever.invoke(question)
        
        # 2. Filter relevant docs
        docs_to_use = []
        for doc in docs:
            res = retrieval_grader.invoke({"question": question, "document": doc.page_content, "question_type": question_type})
            if res.binary_score == 'yes':
                docs_to_use.append(doc)
        
        # 3. Generate answer
        generation = rag_chain.invoke({"documents":format_docs(docs_to_use), "question": question, "question_type": question_type})
        
        # 4. Check hallucination
        hallucination_check = hallucination_grader.invoke({
            "documents": format_docs(docs_to_use), 
            "generation": generation
        })
        
        # 5. Get used documents
        used_docs = doc_lookup.invoke({
            "documents":format_docs(docs_to_use), 
            "question": question, 
            "generation": generation
        })
        end = time.time()
        answer = generation.split("</think>")[-1].strip()
        # 6. Store results
        result = {
            'question': question,
            'question_type': question_type,
            'answer': answer,
            'not_hallucination': hallucination_check.binary_score,
            'process_time': end - start,
            'used_documents': used_docs,
            # Thêm các thông tin khác nếu cần
        }
        results.append(result)
        
    
    # Convert to DataFrame và lưu kết quả
    results_df = pd.DataFrame(results)
    results_df.to_csv('../results/results_reliable_rag.csv', index=False)

csv_file = fr"C:\htN\UIT\learnin\RAG_Techniques\data\alqac_2024\training\EDA\train_alqac_compare.csv"
process_questions(csv_file)